[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/kgweber-cwru/coding-with-ai-wn26/blob/main/week-3-prompt-engineering/assignment.ipynb)

# Week 3 Assignment: Build a Prompt Template Library

## Objective
Create a collection of reusable prompt templates for common tasks in your domain, demonstrating dynamic prompts, few-shot learning, and structured output parsing.

## Requirements
1. Create at least 3 different prompt templates
2. At least one template should use few-shot learning
3. At least one template should produce structured (JSON) output
4. Demonstrate batch processing with one template
5. Include error handling and validation

In [1]:
import os
import sys
from pathlib import Path
import json

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    !pip install -q google-genai google-auth python-dotenv numpy
    from google.colab import auth
    auth.authenticate_user()
    try:
        PROJECT_ID = input("Enter your Google Cloud Project ID (press Enter to use default ADC): ").strip()
    except Exception:
        PROJECT_ID = ""
    if PROJECT_ID:
        os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
else:
    def find_service_account_json(max_up=6):
        p = Path.cwd()
        for _ in range(max_up):
            candidate = p / "series-2-coding-llms" / "creds"
            if candidate.exists():
                for f in candidate.glob("*.json"):
                    return str(f.resolve())
            candidate2 = p / "creds"
            if candidate2.exists():
                for f in candidate2.glob("*.json"):
                    return str(f.resolve())
            p = p.parent
        return None

    sa_path = find_service_account_json()
    if sa_path:
        os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = sa_path
    else:
        try:
            from dotenv import load_dotenv
            load_dotenv()
        except Exception:
            pass

In [2]:
import google.auth
from google import genai
from google.genai import types

creds, project = google.auth.default()
project = os.environ.get("GOOGLE_CLOUD_PROJECT", project)
client = genai.Client(vertexai=True, project=project, location="us-central1")
print(f"Using project: {project}")

print("✓ Environment loaded successfully!")

Using project: coding-with-ai-wn-26
✓ Environment loaded successfully!


## Copy Helper Classes
You'll need these from the concepts notebook:

In [3]:
class PromptTemplate:
    """Reusable prompt template with validation"""
    
    def __init__(self, template, required_vars=None):
        self.template = template
        self.required_vars = required_vars or []
    
    def format(self, **kwargs):
        missing = [var for var in self.required_vars if var not in kwargs]
        if missing:
            raise ValueError(f"Missing required variables: {missing}")
        return self.template.format(**kwargs)
    
    def run(self, **kwargs):
        prompt = self.format(**kwargs)
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=kwargs.get('temperature', 0.3)
            )
        )
        return response.text

class PromptLibrary:
    def __init__(self):
        self.prompts = {}
    
    def register(self, name, template, required_vars=None):
        self.prompts[name] = PromptTemplate(template, required_vars)
    
    def get(self, name):
        if name not in self.prompts:
            raise ValueError(f"Prompt '{name}' not found")
        return self.prompts[name]
    
    def list(self):
        return list(self.prompts.keys())

## Your Prompt Library

### Step 1: Describe Your Domain and Use Cases

**YOUR DESCRIPTION HERE**

My domain: [describe your field]

Common tasks I want to automate:
1. [task 1]
2. [task 2]
3. [task 3]

Why these templates will be useful: [explain]

### Step 2: Create Your Prompt Library

In [ ]:
# Initialize your library
my_library = PromptLibrary()

# Template 1: [Describe what it does]
my_library.register(
    "template_1_name",
    """YOUR TEMPLATE HERE
    Use {variable_name} for dynamic parts
    """,
    required_vars=['list', 'your', 'vars']
)

# Template 2: [Describe what it does]
# YOUR CODE HERE

# Template 3: [Describe what it does]
# YOUR CODE HERE

print(f"Created library with {len(my_library.list())} templates:")
print(my_library.list())

### Step 3: Few-Shot Learning Template
Create a template that uses examples to guide behavior:

In [ ]:
class FewShotTemplate:
    """Template with few-shot examples"""
    
    def __init__(self, task_description, examples):
        self.task_description = task_description
        self.examples = examples  # List of (input, output) tuples
    
    def run(self, new_input):
        # Build prompt with examples
        prompt = f"{self.task_description}\n\n"
        
        for inp, out in self.examples:
            prompt += f"Input: {inp}\nOutput: {out}\n\n"
        
        prompt += f"Input: {new_input}\nOutput:"
        
        response = client.models.generate_content(
            model="gemini-2.5-flash-lite",
            contents=prompt,
            config=types.GenerateContentConfig(
                temperature=0
            )
        )
        
        return response.text

# Create your few-shot template
my_fewshot = FewShotTemplate(
    task_description="YOUR TASK DESCRIPTION",
    examples=[
        ("example input 1", "example output 1"),
        ("example input 2", "example output 2"),
        # Add more examples
    ]
)

# Test it
result = my_fewshot.run("YOUR TEST INPUT")
print(result)

### Step 4: Structured Output Template
Create a template that produces JSON output:

In [ ]:
def extract_structured_info(text, schema):
    """Extract structured information according to schema"""
    
    prompt = f"""Extract information from the text according to this schema.
    
    Schema: {json.dumps(schema, indent=2)}
    
    Text: {text}
    """
    
    response = client.models.generate_content(
        model="gemini-2.5-flash-lite",
        contents=prompt,
        config=types.GenerateContentConfig(
            temperature=0,
            response_mime_type="application/json"
        )
    )
    
    # Parse JSON
    try:
        return json.loads(response.text)
    except json.JSONDecodeError:
        content = response.text
        start = content.find('{')
        end = content.rfind('}') + 1
        if start != -1 and end > start:
            return json.loads(content[start:end])
        raise

# Define your schema
my_schema = {
    "field1": "type",
    "field2": "type",
    # Add your fields
}

# Test with your data
test_text = """YOUR TEST TEXT HERE"""

result = extract_structured_info(test_text, my_schema)
print(json.dumps(result, indent=2))

### Step 5: Batch Processing
Process multiple items with one of your templates:

In [ ]:
def batch_process_with_template(items, template_name, **template_kwargs):
    """Process multiple items using a template from the library"""
    results = []
    template = my_library.get(template_name)
    
    for i, item in enumerate(items, 1):
        print(f"Processing {i}/{len(items)}...", end=" ")
        
        # Prepare kwargs for this item
        item_kwargs = template_kwargs.copy()
        item_kwargs['input_item'] = item  # Adjust this based on your template
        
        result = template.run(**item_kwargs)
        results.append({"input": item, "output": result})
        print("✓")
    
    return results

# Test batch processing
test_items = [
    # YOUR TEST ITEMS
]

batch_results = batch_process_with_template(
    test_items,
    "your_template_name",
    # additional kwargs
)

# Display results
for result in batch_results:
    print(f"\nInput: {result['input']}")
    print(f"Output: {result['output']}")

### Step 6: Error Handling and Validation

In [ ]:
def robust_template_execution(template_func, max_retries=2, **kwargs):
    """Execute template with retry logic"""
    
    for attempt in range(max_retries + 1):
        try:
            result = template_func(**kwargs)
            
            # Add your validation logic here
            # For example: check result is not empty, has expected fields, etc.
            
            return result
            
        except Exception as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            if attempt == max_retries:
                print("Max retries reached, returning None")
                return None
            print("Retrying...")
    
    return None

# Test error handling
# YOUR TEST CODE HERE

## Demonstration

### Show all your templates in action:

In [ ]:
print("=" * 70)
print("PROMPT LIBRARY DEMONSTRATION")
print("=" * 70)

# Demo each template
# YOUR DEMONSTRATION CODE HERE

## Reflection Questions

### 1. Which template was most useful? Why?

**YOUR ANSWER**

### 2. What challenges did you face with structured output parsing?

**YOUR ANSWER**

### 3. How did few-shot examples affect output quality?

**YOUR ANSWER**

### 4. How would you share this library with colleagues?

**YOUR ANSWER**

## Bonus Challenges

### 1. Save/Load Library
Implement methods to save your library to a file and load it later:

In [ ]:
# BONUS CODE HERE

### 2. Template Versioning
Add version tracking to your templates so you can A/B test different prompts:

In [ ]:
# BONUS CODE HERE

### 3. Cost Tracking
Add token counting and cost estimation to your library:

In [ ]:
# BONUS CODE HERE

## Submission Checklist

- [ ] Created at least 3 prompt templates
- [ ] Implemented few-shot learning template
- [ ] Implemented structured output template
- [ ] Demonstrated batch processing
- [ ] Added error handling
- [ ] Tested all templates with realistic examples
- [ ] Answered reflection questions
- [ ] Documented each template's purpose

## Next Week

We'll dive into **embeddings and RAG concepts**:
- Vector representations
- Semantic similarity
- Document search
- Preparing for RAG systems